# Residual Netowrk・Wide Residual Network（CIFAR10を用いた物体認識）


---
## 目的
CIFAR10 Datasetを用いて10クラスの物体認識を行う．

ここでは畳み込みニューラルネットワークのモデルとして，ResNetおよびWideResNetを用いて実験を行う．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


## モジュールのインポート
はじめに必要なモジュールをインポートする．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．


In [ ]:
# モジュールのインポート
from time import time
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込みと確認

学習データ（CIFAR10データセット）を読み込みます．



In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=1),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transform_train, download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transform_test, download=True)

## Residual Network (ResNet)
Residual Network (ResNet) を定義します．

ResNetはBottleneckと呼ばれる構造から構成されています．
まず，`BottleNeck(nn.Module)`で，任意の形のBottleNeckを定義できるクラスを作成します．
`__init__`関数の引数である，`in_planes`は入力される特徴マップのチャンネル数，`planes`はBottleNeck内の特徴マップのチャンネル数を指定します．

また，層を定義する`__init__`内では，`nn.Sequential()`という関数が用いられています．
これは，複数の層が格納されたリストを引数として受け取り，これらの層をひとまとめにしたオブジェクト（層）を定義する関数です・
下の関数では，畳み込みやBatchNormalizationがリスト内にされています．
`nn.Sequential`で定義した層`self.convs`では，実際に演算する際，すなわち`formward()`関数内では，`self.convs(x)`とすることで，リストに格納した演算をその順番通りに処理して返すことができます．

上で定義したBottleNeck構造を活用して，ResNet（ここではResNet50）を定義します．
`ResNet`クラス内で定義されている`self._make_layer()`は，任意の形（総数）のResidual Block (複数のBottleNeck構造からなる層)を定義します．
Residual Blockに入力されるチャンネル数`planes`，BottleNeckの数`num_blocks`，畳み込みのストライド`stride`を指定します．
その後，それらの引数に従い，指定した数・パラメータのBottleNeckをリストないに格納します．
最後に，上で説明した`nn.Sequential`を用いて一塊の層として定義し，返すことで，任意の数の層を持つresidual blockを定義します．

この`_make_layer()`を用いて，`__init__`でResNet全体を定義します．
`AdaptiveAvgPooling()`は任意のサイズの特徴マップに対して平均プーリングを適用する層です．
引数に`(1, 1)`を指定することで，どのようなサイズの特徴マップが入力された場合でも，1×1の特徴マップになるように平均プーリングを行います．

In [ ]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.convs = nn.Sequential(*[nn.Conv2d(in_planes, planes, kernel_size=1, bias=False),
                                     nn.BatchNorm2d(planes),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
                                     nn.BatchNorm2d(planes),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False),
                                     nn.BatchNorm2d(self.expansion * planes)])

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.convs(x)
        out += self.shortcut(x)
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, n_class=10, n_blocks=[3, 4, 6, 3]):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        
        self.res1 = self._make_layer(64, n_blocks[0], stride=1)
        self.res2 = self._make_layer(128, n_blocks[1], stride=2)
        self.res3 = self._make_layer(256, n_blocks[2], stride=2)
        self.res4 = self._make_layer(512, n_blocks[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, n_class)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BottleNeck(self.in_planes, planes, stride))
            self.in_planes = planes * BottleNeck.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        h = self.relu(self.bn1(self.conv1(x)))
        h = self.res1(h)
        h = self.res2(h)
        h = self.res3(h)
        h = self.res4(h)
        h = self.avgpool(h)
        h = torch.flatten(h, 1)
        h = self.fc(h)
        return h
        
        
class ResNet50(ResNet):
    def __init__(self, n_class=10):
        super(ResNet50, self).__init__(n_class, n_blocks=[3, 4, 6, 3])

### 学習

上記で定義したResNetを用いて，ネットワークの学習を行います．
学習のプログラムに関しては，前回までのCIFAR10を用いた演習と同様のため，詳細は割愛します．

In [ ]:
# ネットワークモデル・最適化手法の設定
model_res = ResNet50(n_class=10)
if use_cuda:
    model_res.cuda()

optimizer = torch.optim.SGD(model_res.parameters(), lr=0.01, momentum=0.9)

# ミニバッチサイズ・エポック数の設定
batch_size = 128
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model_res.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model_res(image)
        loss = criterion(y, label)
        
        model_res.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

### テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model_res.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_res(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## Wide Residual Network (WideResNet; WRN)

次に，Wide Residual Network (WideResNet; WRN) を定義します．

WideResNetは，ResNetなどと比べ，ネットワークのフィルタ枚数を増やし（すなわちWideにし），相対的に層を浅くしたネットワークである．

WideResNetでは，ResNetと類似したBlock構造から構成されており，まず初めに基本的なBlock構造（`BasicBlock`）を定義します．
 __init__関数の引数である，in_planesは入力される特徴マップのチャンネル数，planesはBottleNeck内の特徴マップのチャンネル数を指定します．
また，この時，演算の順番が異なることに注意してください．

次に，WideResNetの一部にあたる，`NetworkBlock`モジュールを作成します．
ここでは，前述のnn.Sequential()という関数を活用し，指定した数だけ，BasicBlockを連結したモジュールを作成します．

最後に，これらのモジュールを用いて，WideResNetを作成します．
引数として，ネットワークの深さの`depth`, チャンネル数を増やす割合を示す`widen_factor`を指定します．
`dropRate`はネットワーク内のDropoutの割合を決定するパラメータです．

指定した`depth`および`widen_afctor`から，ネットワーク構造を決定し，作成します．

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes=100, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16 * widen_factor, 32 * widen_factor, 64 * widen_factor]
        assert((depth - 4) % 6 == 0)
        n = int((depth - 4) / 6)
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)

### 学習

上記で定義したWideResNetを用いて，ネットワークの学習を行います．
この時，ネットワークの深さの`depth`とチャンネル数を決定する`wide`を指定し，ネットワークモデルを作成します．

In [ ]:
# ネットワークモデル・最適化手法の設定
depth = 16  # e.g.16,22,28,40
wide  = 2
model_wrn = WideResNet(depth, widen_factor=wide)
if use_cuda:
    model_wrn.cuda()

optimizer = torch.optim.SGD(model_wrn.parameters(), lr=0.01, momentum=0.9)

# ミニバッチサイズ・エポック数の設定
batch_size = 128
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model_wrn.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model_wrn(image)
        loss = criterion(y, label)
        
        model_wrn.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

### テスト

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model_wrn.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_wrn(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## 課題

1. 学習の設定を変更し，認識精度の変化を確認しましょう．
2. WideResNetのパラメータを変更して精度の変化を確認してみましょう．
